In [4]:
import requests
import json
import pandas as pd
import numpy as np
import uuid

pd.options.mode.chained_assignment = None

In [5]:
df_raw

,block_number,miner_reward,fee_recipient_eth_diff,miner,fee_recipient,coinbase_transfers,eth_sent_to_fee_recipient,gas_used,gas_price,effective_priority_fee
0,15759323,28048773889743352,28048773889743352,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0,0,7264048,3861314502,3861314502
1,15759324,23882974671270132,23882974671270132,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0,0,8249554,2895062529,2895062529
2,15759325,15242955454706459,15242955454706459,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0,0,5978322,2549704658,2549704658
3,15759330,36544988792552190,36544988792552190,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0,0,11195943,3264127799,3264127799
4,15759334,28796057392778719,28796057392778719,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,145933890303504,145933890303504,13844136,2080018384,2080018384
...,...,...,...,...,...,...,...,...,...,...
5345,15781692,67820845464526783,67820845464526783,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0,0,12761658,5314422739,5314422739
5346,15781693,51955452072741853,51955452072741853,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0,0,12121900,4286081560,4286081560
5347,15781695,32572886258462427,32572886258462427,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0,0,8883827,3666537659,3666537659
5348,15781696,12208960620680834,12208960620680834,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0,0,909691,13420997482,13420997482


In [3]:
df_raw = pd.read_csv('blocks.csv')
df_raw.sort_values(by='block_number', ascending=True, inplace=True)
df_raw.reset_index(drop=True, inplace=True)
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5350 entries, 0 to 5349
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   block_number               5350 non-null   int64 
 1   miner_reward               5350 non-null   object
 2   fee_recipient_eth_diff     5350 non-null   object
 3   miner                      5350 non-null   object
 4   fee_recipient              5350 non-null   object
 5   coinbase_transfers         5350 non-null   int64 
 6   eth_sent_to_fee_recipient  5350 non-null   int64 
 7   gas_used                   5350 non-null   int64 
 8   gas_price                  5350 non-null   int64 
 9   effective_priority_fee     5350 non-null   int64 
dtypes: int64(6), object(4)
memory usage: 418.1+ KB


In [ ]:
url = f'https://blocks.flashbots.net/v1/blocks?limit=5000'

response = requests.get(url)

raw = response.json()
raw = raw['blocks']

df_raw = pd.DataFrame.from_dict(raw)
df_raw.sort_values(by='block_number', ascending=True, inplace=True)
df_raw.reset_index(drop=True, inplace=True)


def create_transactions(raw):
    txn = []
    for i in range(0, len(raw)):
        txn.append(raw[i]['transactions'])
    
    txn = [item for sublist in txn for item in sublist]
    return txn

txn = create_transactions(raw)
df_txn = pd.DataFrame.from_dict(txn)

df_txn.to_csv('txn.csv', mode='a', index=False)

df_raw.drop(columns=['transactions'])
df_raw.to_csv('blocks.csv', mode='a', index=False)





In [ ]:
temp = pd.read_csv('txn.csv')
temp.drop_duplicates(keep='first', inplace=True)
temp.sort_values(by='block_number', ascending=True, inplace=True)
temp.reset_index(drop=True, inplace=True)


In [ ]:
def create_bundles(df_txn):
    blocks = np.unique(df_txn['block_number'])
    bdl = []
    for i in range(0, len(blocks)):
        bundles = np.unique(df_txn[df_txn['block_number'] == blocks[i]]['bundle_index'])
        for j in range(0, len(bundles)):
            temp = df_txn[(df_txn['block_number'] == blocks[i]) & (df_txn['bundle_index'] == bundles[j])]
            temp['id'] = uuid.uuid4()
            temp.reset_index(drop=True, inplace=True)
            bdl.append(temp.to_dict())

    return bdl

bdl = create_bundles(df_txn)
bdl_df = pd.DataFrame.from_dict(bdl)